# Proyecto 2: GAN
###Tamara Tlaiye Peredo
###166912



In [10]:
#importamos librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gdown
#import time

#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [12]:
#zip
import os
from zipfile import ZipFile

#base de datos
if not os.path.exists("celeba_gan"):
    os.makedirs("celeba_gan")


#link de donde lo sacamos
link = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
output = "celeba_gan/data.zip"
gdown.download(link, output, quiet=True)

#quitamos zip
with ZipFile("celeba_gan/data.zip", "r") as zipobj:
    zipobj.extractall("celeba_gan")

In [ ]:
#!mkdir data_faces && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip

#import zipfile
#with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  #zip_ref.extractall("data_faces/")

In [ ]:
#dataset
datas = keras.utils.image_dataset_from_directory(
    "celeba_gan", label_mode=None, image_size=(64, 64), batch_size=32 #fotos de 64,64
)
datas = datas.map(lambda x: x / 255.0)

Podemos ver que tenemos 202,599 archivos que pertenecen a una clase (en este caso son JPG). 

In [ ]:
#imrpimimos imagen de la dataset
for x in datas:
    plt.axis("off")
    plt.imshow((x.numpy() * 255).astype("int32")[0])
    break

In [ ]:
#discriminator 
#red neuronal layers
disc = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="Discriminator",
)


In [ ]:
#repetimos con generator
latent_dim = 128

#gen
gen = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="Generator",
)


In [ ]:
#clase GAN
class GAN(keras.Model):

    #funcion init
    def __init__(self, disc, gen, latent_dim):
        super().__init__()
        self.disc = disc
        self.gen = gen
        self.latent_dim = latent_dim

    #funcion compile
    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    #funcin metrics
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    #funcion train_step
    def train_step(self, real_images):
        #puntos aleatorios 
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        #imagenes fakes
        generated_images = self.gen(random_latent_vectors)

        #combinamos
        combined_images = tf.concat([generated_images, real_images], axis=0)

        #real o fake
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # puntos aleatorios 
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        #entrenamos el discriminator
        with tf.GradientTape() as tape:
            predictions = self.disc(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.disc.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.disc.trainable_weights)
        )

        #puntos aleatorios 
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        #agregamos etiquetas "todas las imágenes son reales"
        misleading_labels = tf.zeros((batch_size, 1))

        #entrenamos el generator (sin actualizar los pesos del discriminator)
        with tf.GradientTape() as tape:
            predictions = self.disc(self.gen(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.gen.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.gen.trainable_weights))

        #actualizamos las métricas
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [ ]:
#clase GANMonitor
class GANMonitor(keras.callbacks.Callback):
    #funcion init
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim
        
    #funcion onepoch
    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.gen(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("generated_img_%03d_%d.png" % (epoch, i))

In [ ]:
#corremos
epochs = 35

gan = GAN(disc=disc, gen=gen, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    datas, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)